### 이상행동 감지 Project
* 파일 분류
    * 레이블 : 도난 상황 / 일반적인 상황
    * 

### 전처리
* xml -> txt -> yolo {class, x, y, w, h}       + frame
* class / bbox

In [ ]:
import os
import random
import shutil
import ultralytics
import yaml
import cv2
import glob
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from torchvision import transforms
from tqdm import tqdm

In [ ]:
# XML 파일들이 저장된 디렉토리 경로
input_directory = '/path/to/xml/files'
output_directory = '/path/to/output/files'

# 이미지 크기 설정 (예: 1920x1080)
img_width = 1920
img_height = 1080

# XML 파일들을 처리
for filename in os.listdir(input_directory):
    if filename.endswith('.xml'):
        # XML 파일 경로
        xml_file = os.path.join(input_directory, filename)
        
        # XML 파싱
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # YOLO 형식으로 변환된 데이터 저장
        yolo_data = []
        
        # 'track' 태그에서 'label'이 'theft_start' 또는 'theft_end'인 부분 찾기
        for track in root.findall('track'):
            label = track.get('label')
            if label in ['theft_start', 'theft_end']:
                for box in track.findall('box'):
                    xtl = float(box.get('xtl'))
                    ytl = float(box.get('ytl'))
                    xbr = float(box.get('xbr'))
                    ybr = float(box.get('ybr'))

                    # 중심 좌표 계산
                    x_center = (xtl + xbr) / 2.0
                    y_center = (ytl + ybr) / 2.0

                    # 너비와 높이 계산
                    width = xbr - xtl
                    height = ybr - ytl

                    # 상대 좌표 계산
                    x_center /= img_width
                    y_center /= img_height
                    width /= img_width
                    height /= img_height

                    # YOLO 형식: [class_id] [x_center] [y_center] [width] [height]
                    class_id = 0 if label == 'theft_start' else 1
                    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    yolo_data.append(yolo_format)
        
        # 출력 파일명 생성 (예: 원래 파일명이 "example.xml"이라면 "example.txt"로 저장)
        output_filename = os.path.splitext(filename)[0] + '.txt'
        output_file = os.path.join(output_directory, output_filename)
        
        # 결과를 txt 파일로 저장
        with open(output_file, 'w') as file:
            for line in yolo_data:
                file.write(line + '\n')


In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

def xml_to_yolo_bbox(bbox, w, h):
    # YOLO 형식으로 바운딩 박스 변환
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

def process_xml(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 이미지 크기 정보 가져오기 (실제 XML 구조에 맞게 수정 필요)
    width = int(root.find(".//width").text)
    height = int(root.find(".//height").text)

    result = []
    classes = ["theft_start", "theft_end"]  # 클래스 리스트

    for track in root.findall('track'):
        label = track.get('label')
        if label in classes:
            class_id = classes.index(label)
            for box in track.findall('box'):
                if box.get('outside') == '0':
                    xtl = float(box.get('xtl'))
                    ytl = float(box.get('ytl'))
                    xbr = float(box.get('xbr'))
                    ybr = float(box.get('ybr'))
                    
                    bbox = xml_to_yolo_bbox([xtl, ytl, xbr, ybr], width, height)
                    bbox_string = ' '.join([f"{x:.6f}" for x in bbox])
                    result.append(f"{class_id} {bbox_string}")

    # 결과 저장
    if result:
        output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_file))[0] + '.txt')
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(result))

def convert_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    xml_files = [f for f in os.listdir(input_folder) if f.endswith('.xml')]

    for xml_file in tqdm(xml_files, desc="Converting XML to YOLO"):
        process_xml(os.path.join(input_folder, xml_file), output_folder)

# 사용 예
input_folder = '/path/to/xml/folder'
output_folder = '/path/to/output/folder'
convert_folder(input_folder, output_folder)

### YOLO 모델 불러오기

### 학습

### 평가

### 시각화